## Setup
Load initial libraries and config to get things up an running.

In [ ]:
import os
import openai
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

openai.api_key  = os.environ['OPENAI_API_KEY']

vectordb = Chroma(
    persist_directory="../vectors/",
    embedding_function=OpenAIEmbeddings()
)

## Prompt
The prompt to tie together the question and the results from RAG.

In [ ]:
from langchain.prompts import PromptTemplate

template = """Answer the question based on the context below. Keep the answer short. Use two sentences maxiumum. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}

Question: {question}

Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

## Memory
This will be used to track the state of the conversation.

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

## Conversational Retreiver
Set up the LLM to be used for question answering.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

question_answerer = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
    retriever=vectordb.as_retriever(),
    combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT},
    memory=memory
)

## QA Time
Use retrieval augmented generation to look up possible answers from a local vector store and send those to GPT to formulate an answer.

In [ ]:
question = "Can I give a baby a banana?"

result = question_answerer({"question": question})

result["answer"]

In [ ]:
question = "How would I serve it?"

result = question_answerer({"question": question})

result["answer"]

In [ ]:
question = "Is it a choking risk?"

result = question_answerer({"question": question})

result["answer"]